# Preprocessing Data

Er worden voor het dataverhaal 2 datasets gebruikt.

1. De eerste dataset bevat meer dan 278k liedjes van Spotify. De dataset kan gedownload worden van de site:<br>
https://www.kaggle.com/datasets/viktoriiashkurenko/278k-spotify-songs?select=main_dataset.csv<br>
Op de site staan verschillende csv-bestanden. Enkel de main_dataset.csv wordt gebruikt in ons dataverhaal<br>
2. De tweede dataset bevat data over de gemiddelde levensvoldoening per land per jaar. Deze dataset kan gedownload worden van de site:<br>
https://ourworldindata.org/grapher/happiness-cantril-ladder<br>
Er moet hiervoor onder de wereldkaart op de downloadknop geklikt worden om de juiste dataset te downloaden.

De levensvoldoening-dataset is relatief klein en deze kan in oorspronkelijke staat gebruikt worden. De spotify-dataset vergt echter wel nog het een en ander aan preprocessing. De stappen die hiervoor genomen zijn worden hieronder beschreven.

In [2]:
import pandas as pd
import numpy as np

main_dataset = pd.read_csv('main_dataset.csv')

In [3]:
#Deze rijen met het incomplete jaar 2023 worden weggehaald
main_dataset = main_dataset[main_dataset['release_date'].str[:4] != '2023']

#Deze rijen van voor jaar 1960 hebben te weinig data en worden weggehaald
main_dataset = main_dataset[(main_dataset['release_date'].str[:4]).astype(int) >= 1960]

# Deze rijen bevatten Nan values dus worden eruit gehaald
main_dataset = main_dataset.drop(list(main_dataset[main_dataset.isna().any(axis=1)].index), errors='ignore')

# Deze rijen bevatten nul-waarden voor datum
main_dataset = main_dataset.drop([39962, 79501, 79712, 274672], errors='ignore')

In [4]:
#Check de datatypes van de kolommen
for column_name in main_dataset.columns:
  for i in main_dataset[column_name].head(1):
    print(f'{str(column_name):<25} {str(type(i)):<25}')

track_uri                 <class 'str'>            
name                      <class 'str'>            
artists_names             <class 'str'>            
popularity                <class 'int'>            
album_type                <class 'str'>            
is_playable               <class 'bool'>           
release_date              <class 'str'>            
artists_uris              <class 'str'>            
playlist_uris             <class 'str'>            
danceability              <class 'float'>          
energy                    <class 'float'>          
key                       <class 'float'>          
loudness                  <class 'float'>          
mode                      <class 'float'>          
speechiness               <class 'float'>          
acousticness              <class 'float'>          
instrumentalness          <class 'float'>          
liveness                  <class 'float'>          
valence                   <class 'float'>          
tempo       

In [5]:
#Zorg dat sommige kolommen met strings functionele lists worden
for column_name in ['artists_names', 'artists_uris', 'playlist_uris', 'artists_popularities', 'artists_genres', 'artists_followers']:
  main_dataset[column_name] = main_dataset[column_name].map(lambda x: eval(x))

#Zet de kolom met de release dates om in pandas datetime objecten
main_dataset['release_date'] = pd.to_datetime(main_dataset['release_date'])

In [6]:
#Check de datatypes van de kolommen
for column_name in main_dataset.columns:
  for i in main_dataset[column_name].head(1):
    print(f'{str(column_name):<25} {str(type(i)):<25}')

track_uri                 <class 'str'>            
name                      <class 'str'>            
artists_names             <class 'list'>           
popularity                <class 'int'>            
album_type                <class 'str'>            
is_playable               <class 'bool'>           
release_date              <class 'pandas._libs.tslibs.timestamps.Timestamp'>
artists_uris              <class 'list'>           
playlist_uris             <class 'list'>           
danceability              <class 'float'>          
energy                    <class 'float'>          
key                       <class 'float'>          
loudness                  <class 'float'>          
mode                      <class 'float'>          
speechiness               <class 'float'>          
acousticness              <class 'float'>          
instrumentalness          <class 'float'>          
liveness                  <class 'float'>          
valence                   <class 'float

In [7]:
# Controleer welke jaren er nog in de dataset zitten
l = set()
for i in main_dataset['release_date']:
  l = l | {i.year}

print(sorted(list(l)))

[1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


In [8]:
#main_dataset.to_parquet('processed_dataset.parquet')